In [2]:
#%run download-planet-data.py
# Current line count as of Feb 21, 2020
#       4127 confirmed-planets.csv
#     892 k2-candidate-only-planets.csv
#    1236 k2-confirmed-and-candidate-planets.csv
#     431 k2-confirmed-planets.csv
#    2421 kepler-candidate-only-planets.csv
#    4724 kepler-confirmed-and-candidate-planets.csv
#    2357 kepler-confirmed-planets.csv
#      42 tess-confirmed-planets.csv

In [14]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.io import push_notebook, show, output_notebook

# output to notebook
output_notebook()
# output to static HTML file
output_file('tess_example.html')

df = pd.read_csv('data/tess-confirmed-planets.csv')

# iterating the columns 
#for col in df.columns: 
#    print(col) 

 #df[['pl_orbper', 'pl_rade']]  
orb = df.loc[:, 'pl_orbper'] 
rad = df.loc[:, 'pl_rade'] 

#df[['pl_orbper', 'pl_rade']]  
p=figure()
p.scatter(orb, rad)
show(p)

Loading BokehJS ...